# Agent Gateway Quickstart

Agent Gateway is a multi-agent framework that offers native support for Snowflake tools. 

The system can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Agent Gateway does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing the agent.

# Agent Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [1]:
%env LOGGING_LEVEL="DEBUG"

env: LOGGING_LEVEL="DEBUG"


In [1]:
from agent_gateway import Agent
from agent_gateway.tools import CortexSearchTool, CortexAnalystTool, PythonTool, SQLTool
from snowflake.snowpark import Session
import os
from dotenv import load_dotenv

load_dotenv()

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

snowpark = Session.builder.configs(connection_parameters).create()

## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [2]:
search_config = {
    "service_name": "SEC_SEARCH_SERVICE",
    "service_topic": "Snowflake's business,product offerings,and performance.",
    "data_description": "Snowflake annual reports",
    "retrieval_columns": ["CHUNK", "RELATIVE_PATH"],
    "snowflake_connection": snowpark,
    "k": 10,
}

analyst_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "ANALYST",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowflake_connection": snowpark,
}

analyst_trunc_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "ANALYST",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowflake_connection": snowpark,
    "max_results": 5,
}

## Python Tool Configuration

Configuring a Python Tool for the Agent Gateway requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [3]:
import requests


def html_crawl(url):
    response = requests.get(url)
    return response.text


python_crawler_config = {
    "tool_description": "reads the html from a given URL or website",
    "output_description": "html of a webpage",
    "python_func": html_crawl,
}

# SQL Tool Configuration

In [4]:
margin_query = """SELECT 
    LONGNAME,
    SECTOR,
    INDUSTRY,
    CURRENTPRICE,
    MARKETCAP,
    EBITDA,
    CASE 
        WHEN MARKETCAP > 0 THEN (EBITDA * 100.0) / MARKETCAP
        ELSE NULL
    END AS EBITDA_Margin_Percentage
FROM CUBE_TESTING.PUBLIC.SP500;;"""

margin_config = {
    "sql_query": margin_query,
    "connection": snowpark,
    "tool_description": "calculate EBITDA Margin (%) of S&p500 companies",
    "output_description": "ebitda margin (%) metrics per company",
}

## Agent Config

After the tools have been configured, initialize them and configure the agent.

In [19]:
annual_reports = CortexSearchTool(**search_config)
sp500 = CortexAnalystTool(**analyst_config)
web_crawler = PythonTool(**python_crawler_config)
margin_tool = SQLTool(**margin_config)

snowflake_tools = [annual_reports, sp500, web_crawler, margin_tool]
agent = Agent(snowflake_connection=snowpark, tools=snowflake_tools, max_retries=3)

2025-03-20 19:28:53,062 - AgentGatewayLogger - INFO - Cortex Search Tool successfully initialized
2025-03-20 19:28:53,305 - AgentGatewayLogger - INFO - Cortex Analyst Tool successfully initialized
2025-03-20 19:28:53,307 - AgentGatewayLogger - INFO - Python Tool successfully initialized
2025-03-20 19:28:53,309 - AgentGatewayLogger - INFO - SQL Tool successfully initialized
2025-03-20 19:28:53,772 - AgentGatewayLogger - INFO - Cortex gateway successfully initialized


# Agent Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with the Agent Gateway. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

## Structured Data Questions

In [21]:
agent("What is the market cap of Apple?")

2025-03-20 19:29:34,865 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


{'output': "Apple's market cap is approximately $3.02 trillion.",
 'sources': [{'tool_type': 'cortex_analyst',
   'tool_name': 'sp500_semantic_model_cortexanalyst',
   'metadata': [{'Table': 'cube_testing.public.sp500'}]}]}

In [22]:
agent("What is the market cap of Microsoft? and what is its EBITDA margin?")

2025-03-20 19:29:44,827 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-20 19:29:44,833 - AgentGatewayLogger - INFO - running sql_tool task


{'output': 'The market cap of Microsoft is $3,150,184,448,000 and its EBITDA margin is approximately 3.999194%.',
 'sources': [{'tool_type': 'cortex_analyst',
   'tool_name': 'sp500_semantic_model_cortexanalyst',
   'metadata': [{'Table': 'cube_testing.public.sp500'}]},
  {'tool_type': 'sql_tool', 'tool_name': 'sql_tool', 'metadata': [None]}]}

In [23]:
agent("Which company has the bigger EBITDA, Apple or MSFT?")

2025-03-20 19:29:55,732 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-20 19:29:55,736 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


{'output': "Apple has a bigger EBITDA of $129,629,003,776 compared to Microsoft's $125,981,999,104.",
 'sources': [{'tool_type': 'cortex_analyst',
   'tool_name': 'sp500_semantic_model_cortexanalyst',
   'metadata': [{'Table': 'cube_testing.public.sp500'}]}]}

# Unstructured Data Questions

In [24]:
agent("How many customers does Snowflake have?")

2025-03-20 19:30:13,052 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task


{'output': 'As of January 31, 2024, Snowflake had 9,437 total customers.',
 'sources': [{'tool_type': 'cortex_search',
   'tool_name': 'sec_search_service_cortexsearch',
   'metadata': [{'RELATIVE_PATH': '2024_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2023_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2022_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2021_10k_snowflake.pdf'}]}]}

In [7]:
agent(
    "On which platforms can I host Snowflake according to this documentation page https://docs.snowflake.com/en/user-guide/intro-cloud-platforms"
)

2025-03-19 14:23:36,187 - AgentGatewayLogger - INFO - running html_crawl task


{'output': 'Snowflake can be hosted on the following cloud platforms: Amazon Web Services (AWS), Google Cloud Platform (GCP), and Microsoft Azure (Azure).',
 'sources': [{'tool_type': 'custom_tool',
   'tool_name': 'html_crawl',
   'metadata': [None]}]}

## Unstructured + Structured Data Questions

In [25]:
agent(
    "What is the market cap of each of the cloud providers mentioned in Snowflake's annual report?"
)

2025-03-20 19:30:33,090 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task
2025-03-20 19:30:34,195 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-20 19:30:43,287 - AgentGatewayLogger - INFO - Replanning...
2025-03-20 19:30:47,249 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task
2025-03-20 19:30:48,308 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-20 19:30:58,027 - AgentGatewayLogger - INFO - Replanning...
2025-03-20 19:31:02,860 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task
2025-03-20 19:31:03,990 - AgentGatewayLogger - INFO - running summarize task
2025-03-20 19:31:07,524 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


{'output': "The question is unclear because it does not specify which cloud providers are mentioned in Snowflake's annual report. Additionally, there was a technical issue with the request. Please provide more specific details about the cloud providers mentioned in the report or clarify the question.",
 'sources': [{'tool_type': 'cortex_search',
   'tool_name': 'sec_search_service_cortexsearch',
   'metadata': [{'RELATIVE_PATH': '2023_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2024_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2022_10k_snowflake.pdf'},
    {'RELATIVE_PATH': '2021_10k_snowflake.pdf'}]},
  {'tool_type': 'cortex_analyst',
   'tool_name': 'sp500_semantic_model_cortexanalyst',
   'metadata': [{'Table': None}]}]}